# [PROTOTYPE] - Analyze apparent fishing efforts in a region and get vessel details

## Import libraries

In [1]:
import datetime
import os
import logging

import gfwapiclient as gfw

## Instantiate GFW API client

In [2]:
api_base_url = os.getenv("GFW_API_BASE_URL")
api_access_token = os.getenv("GFW_API_ACCESS_TOKEN")

client = gfw.Client(
    api_base_url=api_base_url,
    api_access_token=api_access_token
)

## Retrieve Apparent Fishing Effort in Senegal's EEZ

### Import

In [3]:
from gfwapiclient.resources.fourwings.report.models.request import (
    FourWingsReportDataset,
    FourWingsReportGroupBy,
    FourWingsReportRegion,
    FourWingsReportSpatialResolution,
    FourWingsReportTemporalResolution,
)

### Prepare request parameters

In [4]:
spatial_resolution = FourWingsReportSpatialResolution.HIGH
group_by = FourWingsReportGroupBy.VESSEL_ID
temporal_resolution = FourWingsReportTemporalResolution.MONTHLY
datasets = [FourWingsReportDataset.FISHING_EFFORT_LATEST]
filters = None
# filters = ["geartype in ('TRAWLERS')"]
date_range = "2024-11-01,2025-01-31"
spatial_aggregation = None
geojson = None
region = FourWingsReportRegion(dataset="public-eez-areas", id="8371")

### Fetch 4Wings Report

In [5]:
wings_report = await client.fourwings.aget_report(
    spatial_resolution=spatial_resolution,
    group_by=group_by,
    temporal_resolution=temporal_resolution,
    datasets=datasets,
    filters=filters,
    date_range=date_range,
    spatial_aggregation=spatial_aggregation,
    geojson=geojson,
    region=region,
)

In [6]:
wings_report

In [7]:
len(wings_report.data)

23195

In [8]:
wings_report.data[-1].model_dump()

{'date': '2024-11',
 'detections': None,
 'flag': 'SEN',
 'gear_type': 'TRAWLERS',
 'hours': 2.8513888888888888,
 'vessel_ids': None,
 'vessel_id': 'cee15c9d1-1057-e347-f773-8a64a32bc08a',
 'vessel_type': 'FISHING',
 'entry_timestamp': datetime.datetime(2024, 11, 5, 17, 0, tzinfo=TzInfo(UTC)),
 'exit_timestamp': datetime.datetime(2025, 1, 30, 23, 0, tzinfo=TzInfo(UTC)),
 'first_transmission_date': datetime.datetime(2024, 10, 3, 23, 5, 28, tzinfo=TzInfo(UTC)),
 'last_transmission_date': datetime.datetime(2025, 3, 4, 23, 47, 39, tzinfo=TzInfo(UTC)),
 'imo': '',
 'mmsi': '663113000',
 'call_sign': '6VUM',
 'dataset': 'public-global-vessel-identity:v3.0',
 'report_dataset': 'public-global-fishing-effort:v3.0',
 'ship_name': 'SANTANA',
 'lat': 15.6,
 'lon': -16.829999923706055}

In [9]:
wings_report.data[-2].model_dump()

{'date': '2025-01',
 'detections': None,
 'flag': 'SEN',
 'gear_type': 'TRAWLERS',
 'hours': 0.9930555555555556,
 'vessel_ids': None,
 'vessel_id': 'c6ca30d4c-cca8-92c9-b821-6620ee18940b',
 'vessel_type': 'FISHING',
 'entry_timestamp': datetime.datetime(2024, 11, 2, 18, 0, tzinfo=TzInfo(UTC)),
 'exit_timestamp': datetime.datetime(2025, 1, 30, 23, 0, tzinfo=TzInfo(UTC)),
 'first_transmission_date': datetime.datetime(2017, 3, 30, 15, 41, 45, tzinfo=TzInfo(UTC)),
 'last_transmission_date': datetime.datetime(2025, 3, 4, 23, 59, 34, tzinfo=TzInfo(UTC)),
 'imo': '',
 'mmsi': '663176000',
 'call_sign': '6WGH',
 'dataset': 'public-global-vessel-identity:v3.0',
 'report_dataset': 'public-global-fishing-effort:v3.0',
 'ship_name': 'CARVISA DOS',
 'lat': 12.92,
 'lon': -17.229999542236328}

## Retrieve Vessel Details Using the Vessels API

### Import

In [10]:
from gfwapiclient.resources.vessels.base.enums import (
    VesselDataset,
    VesselInclude,
)

### Prepare request parameters

In [11]:
datasets = [VesselDataset.VESSEL_IDENTITY_LATEST]
includes = [VesselInclude.POTENTIAL_RELATED_SELF_REPORTED_INFO]
ids = [wings_report.data[-1].vessel_id, wings_report.data[-2].vessel_id]

In [12]:
ids

['cee15c9d1-1057-e347-f773-8a64a32bc08a',
 'c6ca30d4c-cca8-92c9-b821-6620ee18940b']

### Fetch Vessels Details

In [13]:
vessels = await client.vessels.aget_vessels(
    ids=ids,
    datasets=datasets,
    includes=includes
)

In [14]:
vessels

In [15]:
len(vessels.data)

2

In [16]:
vessels.data[-1].model_dump()

{'dataset': 'public-global-vessel-identity:v3.0',
 'registry_info_total_records': 1,
 'registry_info': [{'id': 'b224b73fe3580c771dafe8393574e8db',
   'source_code': ['TMT_OTHER'],
   'ssvid': '663113000',
   'flag': 'SEN',
   'ship_name': 'SANTANA',
   'n_ship_name': 'SANTANA',
   'call_sign': '6VUM',
   'imo': None,
   'latest_vessel_info': True,
   'transmission_date_from': datetime.datetime(2024, 10, 4, 15, 51, 33, tzinfo=TzInfo(UTC)),
   'transmission_date_to': datetime.datetime(2025, 2, 20, 16, 3, 42, tzinfo=TzInfo(UTC)),
   'gear_types': ['OTHER_FISHING', 'TRAWLERS'],
   'length_m': 27.1,
   'tonnage_gt': 153.1,
   'vessel_info_reference': '42b59ec9ed000a86e29d4a42c6766bf8',
   'extra_fields': [{'registry_source': 'TMT',
     'iuu_status': None,
     'has_compliance_info': None,
     'images': [],
     'operator': None,
     'builtYear': {'value': None,
      'dateFrom': None,
      'dateFromMask': None,
      'dateTo': None,
      'dateToMask': None},
     'depthM': {'value': No

In [17]:
vessels.data[-2].model_dump()

{'dataset': 'public-global-vessel-identity:v3.0',
 'registry_info_total_records': 0,
 'registry_info': [],
 'registry_owners': [],
 'registry_public_authorizations': [],
 'combined_sources_info': [{'vessel_id': 'dc68427d9-98c8-9ac6-30fb-856306d639ad',
   'gear_types': [{'name': 'TRAWLERS',
     'source': 'COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO',
     'year_from': 2017,
     'year_to': 2017}],
   'ship_types': [{'name': 'FISHING',
     'source': 'COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO',
     'year_from': 2017,
     'year_to': 2017}]},
  {'vessel_id': 'ee41cd32a-a824-f7e8-4a37-539f7f21a539',
   'gear_types': [{'name': 'TRAWLERS',
     'source': 'COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO',
     'year_from': 2018,
     'year_to': 2018}],
   'ship_types': [{'name': 'FISHING',
     'source': 'COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO',
     'year_from': 2018,
     'year_to': 2018}]},
  {'vessel_id': 'c6ca30d4c-cca8-92c9-b821-6620ee18940b',
   'gear_types': [